In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# EQConstraintComp

`EQConstraintComp` is a specialized component that provides a simple way to implement
equality constraints.

You can add one or more outputs to an `EQConstraintComp` that compute the difference
between a pair of input values for the purposes of driving the two inputs to equality. It
computes the output value as:

$$
  name_{output} = \frac{ name_{mult} \times name_{lhs} - name_{rhs} }{f_{norm}(name_{rhs})}
$$

The normalization function $f_{norm}(name_{rhs})$ takes one of the following forms:

$$
  f_{norm}(name_{rhs}) =
  \begin{cases}
    \left| name_{rhs} \right|,      & \text{if normalize and } \left| name_{rhs} \right| \geq 2 \\
    0.25 name_{rhs}^2 + 1,      & \text{if normalize and } \left| name_{rhs} \right| < 2 \\
    1,      & \text{if not normalize}
  \end{cases}
$$ 

The following inputs and outputs are associated with each output variable.

```{eval-rst}
    =========== ======= ====================================================
    Name        I/O     Description
    =========== ======= ====================================================
    {name}      output  output variable
    lhs:{name}  input   left-hand side of difference equation
    rhs:{name}  input   right-hand side of difference equation
    mult:{name} input   left-hand side multiplier of difference equation
    =========== ======= ====================================================
```

The default value for the `rhs:{name}` input can be set to via the
`rhs_val` argument (see arguments below). If the rhs value is fixed (e.g. 0),
then the input can be left unconnected. The `lhs:{name}` input must always have
something connected to it.

The multiplier is optional and will default to 1.0 if not connected.

`EQConstraintComp` supports vectorized outputs. Simply provide a default
value or shape when adding the output that reflects the correct shape.

You can provide the arguments to create an output variable when instantiating an
`EQConstraintComp` or you can use the ``add_eq_output`` method to create one
or more outputs after instantiation.  The constructor accepts all the same arguments
as the ``add_eq_output`` method:

```{eval-rst}
    .. automethod:: openmdao.components.eq_constraint_comp.EQConstraintComp.add_eq_output
       :noindex:
```

Note that the `kwargs` arguments can include any of the keyword arguments normally available
when creating an output variable with the
`add_output` method of a [Component](../../../_srcdocs/packages/core/component).


## Example: Sellar IDF

The following example shows an Individual Design Feasible (IDF) architecture for the
[Sellar](../../../basic_user_guide/multidisciplinary_optimization/sellar.ipynb) problem that demonstrates the use of an `EQConstraintComp`.

In IDF, the direct coupling between the disciplines is removed and the coupling variables
are added to the optimizer’s design variables. The algorithm calls for two new equality
constraints that enforce the coupling between the disciplines. This ensures that the final
optimized solution is feasible, though it is achieved through the optimizer instead of
using a solver.  The two new equality constraints are implemented in this example with
an `EQConstraintComp`.

In [ ]:
import openmdao.api as om


class SellarIDF(om.Group):
    """
    Individual Design Feasible (IDF) architecture for the Sellar problem.
    """
    def setup(self):
        # construct the Sellar model with `y1` and `y2` as independent variables

        self.set_input_defaults('x', 5.)
        self.set_input_defaults('y1', 5.)
        self.set_input_defaults('y2', 5.)
        self.set_input_defaults('z', np.array([2., 0.]))

        self.add_subsystem('d1', SellarDis1withDerivatives(), promotes_inputs=['x', 'z', 'y2'])
        self.add_subsystem('d2', SellarDis2withDerivatives(), promotes_inputs=['y1', 'z'])

        self.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                           x=0., z=np.array([0., 0.])), promotes_inputs=['x', 'z', 'y1', 'y2'])

        self.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes_inputs=['y1'])
        self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes_inputs=['y2'])

        # rather than create a cycle by connecting d1.y1 to d2.y1 and d2.y2 to d1.y2
        # we will constrain y1 and y2 to be equal for the two disciplines

        equal = om.EQConstraintComp()
        self.add_subsystem('equal', equal, promotes_inputs=[('lhs:y1', 'y1'), ('lhs:y2', 'y2')])

        equal.add_eq_output('y1', add_constraint=True)
        equal.add_eq_output('y2', add_constraint=True)

        self.connect('d1.y1', 'equal.rhs:y1')
        self.connect('d2.y2', 'equal.rhs:y2')

        # the driver will effectively solve the cycle
        # by satisfying the equality constraints

        self.add_design_var('x', lower=0., upper=5.)
        self.add_design_var('y1', lower=0., upper=5.)
        self.add_design_var('y2', lower=0., upper=5.)
        self.add_design_var('z', lower=np.array([-5., 0.]), upper=np.array([5., 5.]))
        self.add_objective('obj_cmp.obj')
        self.add_constraint('con_cmp1.con1', upper=0.)
        self.add_constraint('con_cmp2.con2', upper=0.)

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src14", get_code("openmdao.test_suite.components.sellar_feature.SellarIDF"), display=False)

:::{Admonition} `SellarIDF` class definition 
:class: dropdown

{glue:}`code_src14`
:::

In [ ]:
from openmdao.test_suite.components.sellar_feature import SellarIDF

prob = om.Problem(model=SellarIDF())
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', disp=True)
prob.setup()
prob.run_driver();

In [ ]:
print(prob.get_val('x'))

In [ ]:
print([prob.get_val('y1'), prob.get_val('d1.y1')])
print([prob.get_val('y2'), prob.get_val('y2')])

In [ ]:
print(prob.get_val('z'))

In [ ]:
print(prob.get_val('obj_cmp.obj'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('x'), 0., 1e-5)

assert_near_equal([prob.get_val('y1'), prob.get_val('d1.y1')], [[3.16], [3.16]], 1e-5)
assert_near_equal([prob.get_val('y2'), prob.get_val('y2')], [[3.7552778], [3.7552778]], 1e-5)

assert_near_equal(prob.get_val('z'), [1.977639, 0.], 1e-5)

assert_near_equal(prob.get_val('obj_cmp.obj'), 3.18339395045, 1e-5)